In [ ]:
%pip install pandas numpy seaborn matplotlib scikit-learn sentence-transformers

### Importing Libraries ###

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
from preprocessing import clean_data
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

### Loading & Inspecting Data ###


In [ ]:
# Loading Data
delta_df = pd.read_csv("datasets/Delta_Airline_Review_Dataset-Asof02172023.csv")


# Inspecting Data
print("----Data Info----\n")
print(delta_df.info())

print("\n----Data Shape----\n")
print(f'delta_df.shape: {delta_df.shape}')

print("\n----Data Stats----\n")
print(delta_df.describe())

print("\n----Data Null----\n")
print(delta_df.isnull().sum())


### Cleaning Data ###

In [ ]:
cleaned_delta_df = clean_data(delta_df)
print(f'cleaned_delta_df.shape: {cleaned_delta_df.shape}')

### Preliminary Visualization ###

In [ ]:
# Data description distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='sentiment', data=delta_df)
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()


### Preprocessing Data ###

In [ ]:
def tokenize_text(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(text)
    return embeddings

cleaned_delta_df['embedding'] = cleaned_delta_df['reviews'].apply(tokenize_text)

### Model Training ###

In [ ]:
# Features and target
X = np.array(cleaned_delta_df['embedding'].tolist())
y = cleaned_delta_df['sentiment'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification Report:\n{report}")

### Visualizing Model ###

In [ ]:
def plot_decision_tree(model, feature_names, max_depth=3, figsize=(20,10)):
    plt.figure(figsize=figsize)
    # Get a single tree from the forest
    tree = model.estimators_[0]
    plot_tree(tree, 
             feature_names=feature_names,
             max_depth=max_depth,
             filled=True,
             rounded=True)
    plt.show()

def plot_feature_importance(model, feature_names, figsize=(10,6)):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=figsize)
    plt.title('Feature Importance in Delta Airlines Reviews')
    plt.bar(range(len(importances)), 
            importances[indices],
            align='center')
    plt.xticks(range(len(importances)), 
               [feature_names[i] for i in indices], 
               rotation=45,
               ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize feature importance
feature_names = ['Date', 'Traveler Types']  # Your actual feature names
plot_feature_importance(
    model=model,
    feature_names=feature_names
)

# Visualize a single tree from the forest
plot_decision_tree(
    model=model,
    feature_names=feature_names,
    max_depth=3
)